In [67]:
import polars as pl
import pandas as pd
import numpy as np
import os


In [68]:
df = pl.read_parquet("data/data.parquet")
ukazatele = pl.read_csv("data/cis_ukazatelu.csv")
uzemi = pl.read_csv("data/cis_uzemi.csv")
df.head()

rok,kodukaz,koduzemi,hodnota
i32,i32,i32,f32
2021,10000,500011,0.0
2021,10000,500020,0.0
2021,10000,500046,0.0
2021,10000,500062,0.0
2021,10000,500071,0.0


In [69]:
# save yearly ukaz stats by uzemi
years_enum = df["koduzemi"].unique()
matrix_alluzemi = df.pivot(values="hodnota", index=("rok", "koduzemi"), columns="kodukaz", aggregate_function=None).lazy()
matrices = {}
matrixes_uzemi = pl.collect_all([matrix_alluzemi.filter(pl.col("koduzemi") == uzemi) for uzemi in years_enum])
matrixes_uzemi = {m.lazy().select(pl.col("koduzemi")).head(1).collect()[0, 0]: m.drop("koduzemi") for m in matrixes_uzemi}

In [70]:
try:
    os.mkdir("data/yearlystatsbyuzemi")
except:
    pass
for uzemi in matrixes_uzemi.keys():
    matrixes_uzemi[uzemi].write_ipc(f"data/yearlystatsbyuzemi/{uzemi}.arrow", compression="uncompressed")

In [71]:
# save uzemi ukaz stats by year
years_enum = df["rok"].unique()
matrix_alluzemi = df.pivot(values="hodnota", index=("rok", "koduzemi"), columns="kodukaz", aggregate_function=None).lazy()
matrixes_yearly = {}
matrixes_yearly = pl.collect_all([matrix_alluzemi.filter(pl.col("rok") == rok) for rok in years_enum])
matrixes_yearly = {m.lazy().select(pl.col("rok")).head(1).collect()[0, 0]: m.drop("rok") for m in matrixes_yearly}

In [72]:
try:
    os.mkdir("data/uzemnistatsbyyears")
except:
    pass
for year in matrixes_yearly.keys():
    matrixes_yearly[year].write_ipc(f"data/uzemnistatsbyyears/{year}.arrow", compression="uncompressed")

In [73]:
def get_yearly_matrices(year):
    lf = pl.scan_ipc(f"data/uzemnistatsbyyears/{str(year)}.arrow", memory_map=True)
    return lf
def get_uzemi_matrices(uzemi):
    lf = pl.scan_ipc(f"data/yearlystatsbyuzemi/{str(uzemi)}.arrow", memory_map=True)
    return lf

In [95]:
df = get_yearly_matrices(2019).collect().to_pandas().dropna(axis=1, how="all")
pocet_obyvatel = df[["110940", "110950", "110970", "110980", "110990","110930"]].set_index(df["koduzemi"])
pocty = pocet_obyvatel.sum(axis=1)
pocty

koduzemi
500011    1869.0
500020    1220.0
500046    1710.0
500062    2019.0
500071    1753.0
           ...  
599930    2701.0
599948    3464.0
599956    1836.0
599964     954.0
599999    2697.0
Length: 6258, dtype: float32